# Double Integrator Motion Simulation

The goal of this exercise is to build a Python class that defines the dynamics of a double integrator.

Unlike in the `turtlebot_dynamics.ipynb` notebook, here we will leverage *vectorization* to rollout several trajectories simultaneously.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)

## Double Integrator Dynamics

The double integrator dynamics are:
\begin{equation}
\dot{x} = v_x, \quad \dot{y} = v_y, \quad \dot{v}_x = a_x, \quad \dot{v}_y = a_y.
\end{equation}
where $(x,y)$ is the position, $(v_x, v_y)$ is the velocity, and $(a_x, a_y)$ is the acceleration control input.

#### Exercise: Define Double Integrator Dynamics Class Using Vectorization

Complete the implementations of the `feed_forward` and `rollout` functions. Note that in the `feed_forward` method you should leverage the variables `A_stack` and `B_stack` to define the vectorized equation to compute multiple trajectories in parallel.

In [ ]:
from dynamics import Dynamics
class DoubleIntegratorDynamics(Dynamics):

  def __init__(self) -> None:
    super().__init__()
    self.xdd_max = 0.5 # m/s^2
    self.ydd_max = 0.5 # m/s^2
    self.n = 4
    self.m = 2

  def feed_forward(self, state:np.array, control:np.array):

    num_rollouts = int(state.shape[0] / self.n)

    # Define Gaussian Noise
    if self.noisy == False:
      var = np.array([0.0, 0.0, 0.0, 0.0])
    elif self.noisy == True:
      var = np.array([0.01, 0.01, 0.001, 0.001])

    var_stack = np.tile(var, (num_rollouts))
    w = np.random.normal(loc=np.zeros(state.shape), scale=var_stack)

    # State space dynamics
    A = np.array([[1.0, 0.0, self.dt, 0.0],
                  [0.0, 1.0, 0.0, self.dt],
                  [0.0, 0.0, 1.0, 0.0],
                  [0.0, 0.0, 0.0, 1.0]])

    B = np.array([[0.0, 0.0],
                  [0.0, 0.0],
                  [self.dt, 0.0],
                  [0.0, self.dt]])

    # Stack to parallelize trajectories
    A_stack = np.kron(np.eye(num_rollouts), A)
    B_stack = np.tile(B, (num_rollouts, 1))

    ############################## Code starts here ##############################
    """
    Construct "state_new" by applying discrete time dynamics vectorized equations. Will require use of "A_stack" and "B_stack".
    """

    ############################## Code ends here ##############################

    # Add noise
    state_new = state_new + w

    return state_new

  def rollout(self, state_init, control_traj, num_rollouts):

    num_steps = control_traj.shape[1]

    state_traj = np.zeros((self.n*num_rollouts, num_steps+1))
    state_traj[:,0] = np.tile(state_init, num_rollouts)
    ############################## Code starts here ##############################
    """
    Populate "state_traj" using only one for-loop, along with the "feed_forward" function above.
    """

    ############################## Code ends here ##############################

    return state_traj


Run the script below to simulate the TurtleBot motion and plot the state and control trajectories.

In [ ]:
# Run Double Integrator Dynamics Rollouts

# Constants
num_rollouts = 10
num_steps = 100

# Define the dynamics class
di_dynamics = DoubleIntegratorDynamics()

# Define the control inputs
control_traj = np.zeros((di_dynamics.m,num_steps))
control_traj[0,:] = np.sin(np.linspace(0.0, 2*np.pi, num_steps))
control_traj[1,:] = np.cos(np.linspace(0.0, 2*np.pi, num_steps))

# Define the initial state
state_init = np.array([0.0, 0.0, 0.0, 0.0])

# Rollouts dynamics
state_traj_rollouts = di_dynamics.rollout(state_init, control_traj, num_rollouts)

# Plot control inputs
fig_ctrl, axs_ctrl = plt.subplots(nrows=di_dynamics.m)
ylabels_ctrl = ["xdd", "ydd"]
ylims_ctrl = np.array([[-1.1, 1.1],[-1.1, 1.1]])
for j in range(di_dynamics.m):
  axs_ctrl[j].plot(np.linspace(0,num_steps,num_steps), control_traj[j, :])
  axs_ctrl[j].set_ylabel(ylabels_ctrl[j])
  axs_ctrl[j].set_ylim(ylims_ctrl[j,:])
fig_ctrl.suptitle("Control")

# Plots state trajectory rollouts
fig_state, axs_state = plt.subplots(nrows=di_dynamics.n)
ylabels_state = ["x", "y", "xd", "yd"]
ylims_state = np.array([[-0.5, 0.5], [-0.5, 0.5], [-0.1, 0.5], [-0.3, 0.3]])
for i in range(num_rollouts):
  for j in range(di_dynamics.n):
    axs_state[j].plot(np.linspace(0,num_steps+1,num_steps+1), state_traj_rollouts[di_dynamics.n*i + j, :], c='c')
    axs_state[j].set_ylabel(ylabels_state[j])
    axs_state[j].set_ylim(ylims_state[j,:])
fig_state.suptitle("State")
plt.show()